In [7]:
%load_ext autoreload
%autoreload 2

In [1]:
import numpy as np
from sweep.experiment import Experiment, ExperimentType
from model.data import DataModel, KFeaturesDefinition, k_features_factory
from erm.problems.problems import ProblemType
import subprocess


Define a datamodel

In [2]:
d = 100
vanilla_gaussian_diagonal = [(1, d)]
x_diagonal = KFeaturesDefinition(diagonal=vanilla_gaussian_diagonal)
θ_diagonal = KFeaturesDefinition(diagonal=vanilla_gaussian_diagonal)
ω_diagonal = KFeaturesDefinition(diagonal=vanilla_gaussian_diagonal)
δ_diagonal = KFeaturesDefinition(diagonal=vanilla_gaussian_diagonal)
ν_diagonal = KFeaturesDefinition(diagonal=vanilla_gaussian_diagonal)

k_features_kwargs = {
    "x_diagonal": x_diagonal,
    "θ_diagonal": θ_diagonal,
    "ω_diagonal": ω_diagonal,
    "δ_diagonal": δ_diagonal,
    "ν_diagonal": ν_diagonal,
}

data_model = DataModel(
    d,
    normalize_matrices=False,
    data_model_factory=k_features_factory,
    factory_kwargs=k_features_kwargs,
    name="vanilla-gaussian"
)

Define an experiment

In [3]:
experiment = Experiment(
    state_evolution_repetitions=1,
    erm_repetitions=1,
    alphas=np.linspace(0.01,10,5),
    epsilons=np.array([0, 0.1]),
    lambdas=np.array([0.1]),
    taus=np.array([0.1]),
    d=d,
    experiment_type=ExperimentType.Sweep,
    data_model = data_model,
    test_against_epsilons=np.array([0,0.1]),
    erm_problem_type=ProblemType.Logistic,
    gamma_fair_error=0.01,
    name="vanilla-gaussian"
)
experiment_json = experiment.to_json()

In [4]:
with open("experiment.json", "w") as f:
    f.write(experiment_json)

In [5]:
command = ["pwd"]
subprocess.run(command)

/Users/kasimirtanner/Documents/LocalCode/Code_PDM/AISTATS


CompletedProcess(args=['pwd'], returncode=0)

In [6]:
venv_python = ".venv/bin/python"
command = ["mpiexec", "-n", "5", venv_python, "sweep/run_sweep.py", "--json", experiment_json, "--log-level", "INFO"]
subprocess.run(command)

INFO:root:The MPI comm size is 5
INFO:root:This process has rank 0
INFO:root:The MPI comm size is 5
INFO:root:The MPI comm size is 5
INFO:root:This process has rank 4
INFO:root:Starting Master for experiment vanilla-gaussian
INFO:root:Starting Experiment vanilla-gaussian
INFO:root:This process has rank 3
INFO:root:The MPI comm size is 5
INFO:root:The MPI comm size is 5
INFO:root:This process has rank 2
INFO:root:This process has rank 1
  0%|          | 0/20 [00:00<?, ?it/s]INFO:root:Starting all processes
INFO:root:Sending task 0 to 1
INFO:root:Sending task 1 to 2
INFO:root:Sending task 2 to 3
INFO:root:Sending task 3 to 4
INFO:root:All processes started - receiving results and sending new tasks
INFO:root:Received the 0th task
INFO:root:Starting task 1
INFO:root:Initializing data model
INFO:root:Starting task 3
INFO:root:Starting task 2
INFO:root:Initializing data model
INFO:root:Initializing data model
INFO:root:Starting task 4
INFO:root:Initializing data model


2025-02-17 20:48:23,925 - Rank 0 - INFO - The MPI comm size is 5
2025-02-17 20:48:23,926 - Rank 0 - INFO - This process has rank 0
2025-02-17 20:48:23,926 - Rank 0 - INFO - Starting Master for experiment vanilla-gaussian
2025-02-17 20:48:23,926 - Rank 0 - INFO - Starting Experiment vanilla-gaussian
2025-02-17 20:48:23,936 - Rank 0 - INFO - Starting all processes
2025-02-17 20:48:23,937 - Rank 0 - INFO - Sending task 0 to 1
2025-02-17 20:48:23,939 - Rank 0 - INFO - Sending task 1 to 2
2025-02-17 20:48:23,939 - Rank 0 - INFO - Sending task 2 to 3
2025-02-17 20:48:23,939 - Rank 0 - INFO - Sending task 3 to 4
2025-02-17 20:48:23,939 - Rank 0 - INFO - All processes started - receiving results and sending new tasks
2025-02-17 20:48:23,939 - Rank 0 - INFO - Received the 0th task


INFO:root:d: 100
INFO:root:normalize_matrices: False
INFO:root:d: 100
INFO:root:normalize_matrices: False
INFO:root:Ratio of first to last eigenvalue of Σ_x: 1.0
INFO:root:Norm Σ_x: 100.0
INFO:root:Norm Σ_ω: 100.0
INFO:root:Norm Σ_δ: 100.0
INFO:root:Norm Σ_ν: 100.0
INFO:root:Σ_x: [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
INFO:root:Σ_ω: [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
INFO:root:Σ_δ: [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
INFO:root:Σ_ν: [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
INFO:root:Σ_x value counts: (array([0., 1.]), arra

2025-02-17 20:48:25,538 - Rank 0 - INFO - Received task 2 from 2
2025-02-17 20:48:25,538 - Rank 0 - INFO - Saved Task(id=2, task_type=<TaskType.SE: 1>, erm_problem_type=None, alpha=0.01, epsilon=0.1, test_against_epsilons=[0.0, 0.1], lam=0.1, tau=0.1, d=100, data_model=DataModel(d=100, normalize_matrices=False, data_model_factory=k_features_factory, factory_kwargs={'x_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'θ_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'ω_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'δ_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'ν_diagonal': KFeaturesDefinition(diagonal=[[1, 100]])}, name='vanilla-gaussian', description='', gamma=1), values={}, gamma_fair_error=0.01, gamma=1, result=<sweep.results.state_evolution.SEResult object at 0x15cde7bd0>)
2025-02-17 20:48:25,539 - Rank 0 - INFO - Received the 1th task
2025-02-17 20:48:25,638 - Rank 0 - INFO - Received task 1 from 1
2025-02-17 20:48:25,652 - Rank 0 - INFO - Saved Task(id=1

INFO:root:d: 100
INFO:root:normalize_matrices: False
INFO:root:Ratio of first to last eigenvalue of Σ_x: 1.0
INFO:root:Norm Σ_x: 100.0
INFO:root:Norm Σ_ω: 100.0
INFO:root:Norm Σ_δ: 100.0
INFO:root:Norm Σ_ν: 100.0
INFO:root:Σ_x: [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
INFO:root:Σ_ω: [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
INFO:root:Σ_δ: [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
INFO:root:Σ_ν: [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
INFO:root:Σ_x value counts: (array([0., 1.]), array([9900,  100]))
INFO:root:Σ_ω value counts: (array([

...

2025-02-17 20:48:40,898 - Rank 0 - INFO - Received task 18 from 1
2025-02-17 20:48:40,898 - Rank 0 - INFO - Saved Task(id=18, task_type=<TaskType.ERM: 0>, erm_problem_type=<ProblemType.Logistic: 0>, alpha=7.5024999999999995, epsilon=0.1, test_against_epsilons=[0.0, 0.1], lam=0.1, tau=0.1, d=100, data_model=DataModel(d=100, normalize_matrices=False, data_model_factory=k_features_factory, factory_kwargs={'x_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'θ_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'ω_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'δ_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'ν_diagonal': KFeaturesDefinition(diagonal=[[1, 100]])}, name='vanilla-gaussian', description='', gamma=1), values={}, gamma_fair_error=0.01, gamma=1, result=<sweep.results.erm.ERMResult object at 0x15cdfe490>)
2025-02-17 20:48:40,898 - Rank 0 - INFO - Received the 18th task


INFO:root:Finished ERM Task(id=19, task_type=<TaskType.ERM: 0>, erm_problem_type=<ProblemType.Logistic: 0>, alpha=10.0, epsilon=0.0, test_against_epsilons=[0.0, 0.1], lam=0.1, tau=0.1, d=100, data_model=DataModel(d=100, normalize_matrices=False, data_model_factory=k_features_factory, factory_kwargs={'x_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'θ_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'ω_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'δ_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'ν_diagonal': KFeaturesDefinition(diagonal=[[1, 100]])}, name='vanilla-gaussian', description='', gamma=1), values={}, gamma_fair_error=0.01, gamma=1, result=None)
INFO:root:Received task 19 from 3
INFO:root:Saved Task(id=19, task_type=<TaskType.ERM: 0>, erm_problem_type=<ProblemType.Logistic: 0>, alpha=10.0, epsilon=0.0, test_against_epsilons=[0.0, 0.1], lam=0.1, tau=0.1, d=100, data_model=DataModel(d=100, normalize_matrices=False, data_model_factory=k_features_factory,

2025-02-17 20:48:41,929 - Rank 0 - INFO - Received task 19 from 3
2025-02-17 20:48:41,929 - Rank 0 - INFO - Saved Task(id=19, task_type=<TaskType.ERM: 0>, erm_problem_type=<ProblemType.Logistic: 0>, alpha=10.0, epsilon=0.0, test_against_epsilons=[0.0, 0.1], lam=0.1, tau=0.1, d=100, data_model=DataModel(d=100, normalize_matrices=False, data_model_factory=k_features_factory, factory_kwargs={'x_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'θ_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'ω_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'δ_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'ν_diagonal': KFeaturesDefinition(diagonal=[[1, 100]])}, name='vanilla-gaussian', description='', gamma=1), values={}, gamma_fair_error=0.01, gamma=1, result=<sweep.results.erm.ERMResult object at 0x15cdfe9d0>)
2025-02-17 20:48:41,929 - Rank 0 - INFO - Received the 19th task
2025-02-17 20:48:42,102 - Rank 0 - INFO - Received task 20 from 4
2025-02-17 20:48:42,102 - Rank 0 - INFO - 

INFO:root:Marking experiment vanilla-gaussian as finished
INFO:root:Experiment took 18 seconds
100%|██████████| 20/20 [00:18<00:00,  1.10it/s]
INFO:root:All done - signaling exit
INFO:root:Received exit signal - my rank is 4
INFO:root:Worker exiting - my rank is 4
INFO:root:Received exit signal - my rank is 2
INFO:root:Worker exiting - my rank is 2
INFO:root:Received exit signal - my rank is 1
INFO:root:Worker exiting - my rank is 1
INFO:root:Received exit signal - my rank is 3
INFO:root:Worker exiting - my rank is 3


CompletedProcess(args=['mpiexec', '-n', '5', '.venv/bin/python', 'sweep/run_sweep.py', '--json', '{"name": "vanilla-gaussian", "state_evolution_repetitions": 1, "erm_repetitions": 1, "alphas": [0.01, 2.5075, 5.005, 7.5024999999999995, 10.0], "epsilons": [0.0, 0.1], "test_against_epsilons": [0.0, 0.1], "lambdas": [0.1], "taus": [0.1], "d": 100, "experiment_type": "Sweep", "data_model": {"d": 100, "normalize_matrices": false, "data_model_factory_name": "k_features_factory", "factory_kwargs": {"x_diagonal": {"diagonal": [[1, 100]]}, "\\u03b8_diagonal": {"diagonal": [[1, 100]]}, "\\u03c9_diagonal": {"diagonal": [[1, 100]]}, "\\u03b4_diagonal": {"diagonal": [[1, 100]]}, "\\u03bd_diagonal": {"diagonal": [[1, 100]]}}, "name": "vanilla-gaussian", "description": "", "gamma": 1}, "erm_problem_type": "Logistic", "gamma_fair_error": 0.01}', '--log-level', 'INFO'], returncode=0)